In [1]:
# AXILiteS
# 0x0000 : Control signals
#          bit 0  - ap_start (Read/Write/COH)
#          bit 1  - ap_done (Read/COR)
#          bit 2  - ap_idle (Read)
#          bit 3  - ap_ready (Read)
#          bit 7  - auto_restart (Read/Write)
#          others - reserved
# 0x0004 : Global Interrupt Enable Register
#          bit 0  - Global Interrupt Enable (Read/Write)
#          others - reserved
# 0x0008 : IP Interrupt Enable Register (Read/Write)
#          bit 0  - Channel 0 (ap_done)
#          bit 1  - Channel 1 (ap_ready)
#          others - reserved
# 0x000c : IP Interrupt Status Register (Read/TOW)
#          bit 0  - Channel 0 (ap_done)
#          bit 1  - Channel 1 (ap_ready)
#          others - reserved
# 0x1000 : Data signal of num_nonzero_symbols
#          bit 31~0 - num_nonzero_symbols[31:0] (Read)
# 0x1004 : Control signal of num_nonzero_symbols
#          bit 0  - num_nonzero_symbols_ap_vld (Read/COR)
#          others - reserved
# 0x0400 ~
# 0x07ff : Memory 'symbol_histogram_value_V' (256 * 32b)
#          Word n : bit [31:0] - symbol_histogram_value_V[n]
# 0x0800 ~
# 0x0bff : Memory 'symbol_histogram_frequency_V' (256 * 32b)
#          Word n : bit [31:0] - symbol_histogram_frequency_V[n]
# 0x0c00 ~
# 0x0fff : Memory 'encoding_V' (256 * 32b)
#          Word n : bit [31:0] - encoding_V[n]
# (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)

from pynq import Overlay

overlay = Overlay("../bitstream/huffman-encoding-test.bit")
overlay.download()

huffman_encoding = overlay.huffman_encoding_0

symbol_table = [
    { 'symbol': 'a', 'freq': 3 },
    { 'symbol': 'b', 'freq': 1 },
    { 'symbol': 'c', 'freq': 2 },
    { 'symbol': 'd', 'freq': 5 },
    { 'symbol': 'e', 'freq': 5 },
    { 'symbol': 'f', 'freq': 1 },
    { 'symbol': 'g', 'freq': 0 },
]

for idx, sym in enumerate(symbol_table):
    # write symbol & frequency
    huffman_encoding.write(0x0400 + 4*idx, ord(sym['symbol']))
    huffman_encoding.write(0x0800 + 4*idx, sym['freq'])
    
# start
huffman_encoding.write(0x0000, 1)

from time import sleep

sleep(1)

# read number of symbols
num = huffman_encoding.read(0x1000)
print('There are ' + str(num) + ' symbols in huffman tree:')

# read encoding
get_bin = lambda x, n: format(x, 'b').zfill(n)
for idx, sym in enumerate(symbol_table):
    encoding = huffman_encoding.read(0x0c00 + 4*ord(sym['symbol']))
    print('symbol : ' + sym['symbol'] + ', code word : ' + get_bin(encoding, 32))

There are 6 symbols in huffman tree:
symbol : a, code word : 00000000000000000000000000000010
symbol : b, code word : 00000000000000000000000011100100
symbol : c, code word : 00000000000000000000000001100011
symbol : d, code word : 00000000000000000000000001000010
symbol : e, code word : 00000000000000000000000000100010
symbol : f, code word : 00000000000000000000000111100100
symbol : g, code word : 00000000000000000000000000000000
